In [1]:
!pip install ta
!pip install scikit-learn scikit-optimize xgboost lightgbm catboost

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=a528164a82111cb70855e6c472f2d07173af3855c123e2f4b0115205e9310f4a
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.1 MB/s eta 0:00:00


In [ ]:
import ast
file_result = "CADJPY case study3.txt"

with open(file_result, "r") as f:
  result_casestudy = f.readlines()

diff_or_pct = 0
upper_rate, lower_rate = 60,40

profit_case_month_1= {}
profit_case_month_2 = {}

predict_case_month_1 = {}
predict_case_month_2 = {}
symbol =  None
parameter = {}

for i, line in enumerate(result_casestudy):
    line = line.replace('\n', '')
    if "Case" in line:
      temp = line.split(":")[1]
      symbol = temp.split(",")[0].strip()
      if temp.split(",")[1] == "diff()":
        diff_or_pct = 1
      else:
        diff_or_pct = 2
      upper_rate, lower_rate = int(temp.split(",")[2]), int(temp.split(",")[3])
    if "parameter" in line:
      # Xử lý chuỗi để chuyển đổi thành từ điển

      para_LGBM = result_casestudy[i+1].split(":")[1].strip()
      dict_string = para_LGBM.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["LGBM"] = parameter_dict

      para_LGBM2= result_casestudy[i+2].split(":")[1].strip()
      dict_string = para_LGBM2.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["LGBM2"] = parameter_dict

      para_XGB = result_casestudy[i+3].split(":")[1].strip()
      dict_string = para_XGB.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["XGB"] = parameter_dict


      para_RF = result_casestudy[i+4].split(":")[1].strip()
      dict_string = para_RF.replace("OrderedDict", "")
      parameter_dict = ast.literal_eval(dict_string)
      parameter["RF"] = parameter_dict


FileNotFoundError: [Errno 2] No such file or directory: 'CADJPY case study3.txt'

In [ ]:
# Tổng quan bước này ta lấy dữ liệu từ mt5
# Sau đó lưu vào biến tên là data_forex
# Và nhớ khai báo các cặp tiền tệ cần thiết vào biến symbols

import requests
import pandas as pd
from datetime import datetime
import os
"datetime.now().date().isoformat()"
"""Cảnh báo không được thay đổi địa chỉ ip của hàm này"""
def get_history(symbol: str, timeframe: str = "D1", start: str = "2015-01-01", end: str = "2024-04-12"):
    params = {
        "timeframe": timeframe,
        "start": start,
        "end": end,
    }
    response = requests.get(f"http://221.132.33.180:8005/history/{symbol}", params=params)
    response.raise_for_status()
    data = response.json()
    data = pd.DataFrame(data)
    data["Date"] = pd.to_datetime(data['Date'], unit='s')
    data.set_index("Date", inplace=True)
    return data


# symbols = {'all': ["EURCHF", "EURNZD", "NZDUSD", "AUDNZD", "USDJPY", "NZDJPY", "GBPJPY", "USDCHF"]}


"""Đoạn code này có thể chỉnh sửa lấy loại tiền tệ theo yêu cầu"""
def get_data(symbol):
  data_forex = get_history(symbol)
  return data_forex


symbol = "USDCHF"
# """   Chổ này là chổ cần chỉnh sửa nếu muốn đi kiểm tra tiền tệ"""
# """--------------------------------------------------------------------------------"""

# # diff_or_pct = 2
# diff_or_pct = 1
# # Biến diff_or_pct chỉ có hai giá trị là 1 và 2
# """Ở đây nếu ta chọn 1 là sẽ dùng cách diff và chọn 2 là dùng pct_change"""
# # upper_rate, lower_rate = 60,40
# upper_rate, lower_rate = 65,35

"""Chỉ cần thay đổi diff_or_pct và upper_rate và lower_rate, symbol để thay đổi mã cần thiết"""
# symbol_file = symbol+".csv"

# if os.path.exists(symbol_file):
#     print("File CSV đã tồn tại")

# else:
#     data= get_data(symbol)
#     data.to_csv(symbol_file)

data= get_data(symbol)

In [ ]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2015-01-02,0.99289,1.00183,0.99265,1.00142,87310
2015-01-05,1.00632,1.01124,1.00319,1.00677,109808
2015-01-06,1.00676,1.01084,1.00403,1.01002,113132
2015-01-07,1.01001,1.01745,1.00935,1.01423,123995
2015-01-08,1.01423,1.02157,1.01358,1.01827,99502
...,...,...,...,...,...
2024-04-05,0.90129,0.90703,0.89972,0.90217,59978
2024-04-08,0.89997,0.90648,0.89971,0.90526,54061
2024-04-09,0.90530,0.90557,0.90122,0.90341,46356


In [ ]:
import pandas as pd
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy

# Tải dữ liệu
file_path = '/content/'
data_ori = pd.read_csv(file_path+symbol_file)
data_ori  = data_ori.rename(columns = {"Date":"DATE", "Open": "OPEN", "High": "HIGH", "Low": "LOW","Close": "CLOSE", "Volume": "VOLUME"  })

# Gán các cột cho dễ nhìn hơn
data_ori = ta.add_all_ta_features(
    df=data_ori,
    open="OPEN",
    high="HIGH",
    low="LOW",
    close="CLOSE",
    volume="VOLUME",
    fillna=True
)

In [ ]:
data_ori

In [ ]:
import numpy as np
data = data_ori.copy()
data['DATE'] = pd.to_datetime(data['DATE'])
# Giả sử 'data' là DataFrame của bạn chứa cột 'CLOSE'
data.reset_index(inplace=True)
time_test = 2
time_split = data['DATE'].max() - pd.DateOffset(months=time_test)
data_smooth = data[data['DATE'] <= time_split].copy()
# data_smooth['CLOSE_MA'] = data_smooth['CLOSE'].rolling(window=5).mean()
"""Chổ này là chổ cần được chỉnh sửa lại theo pct_change hay là diff"""
if diff_or_pct == 1:

  data_smooth['pct_change'] = data_smooth['CLOSE'].diff()
if diff_or_pct ==2:
  data_smooth['pct_change'] = data_smooth['CLOSE'].pct_change()

#pct change là tính theo phần trăm lợi nhuận
"""-----------------------------------------------------------------"""

# CHIA Theo phần trăm là xaidf pct_change, thứ hai là chia theo chênh lệch diff()
# 10 +12 12-10/10 =0.2 này pct_change(),  10  ngày 12 thì lời 2 đồng này là  diff()
# 100 102 2%
data_smooth['pct_change_ma'] = data_smooth['pct_change'].rolling(window=1).mean()

data_smooth['pct_change_ma'] = data_smooth['pct_change_ma'].shift(-1)
data_smooth = data_smooth.dropna()
# Giả sử data đã có cột 'pct_change'
# Xác định ngưỡng dựa trên phân phối hoặc độ lệch chuẩn


"""Chổ này là chổ cần chỉnh sửa theo tỷ lệ hiện ta có hai tỷ lệ là 65 35 và    60 40"""
threshold_upper = np.percentile(data_smooth['pct_change_ma'].dropna(), upper_rate)
threshold_lower = np.percentile(data_smooth['pct_change_ma'].dropna(), lower_rate) #quan tâm số dưới


""" -------------------------------------------------------------------------------- """

 # Nhãn là 1 0 -1 với 1 là tăng, 0 là không làm gì hết và -1 là giảm.


# Gán nhãn dựa trên ngưỡng

data_smooth['label'] = np.select(
    [
        data_smooth['pct_change_ma'] <= threshold_lower,
        data_smooth['pct_change_ma'] >= threshold_upper
    ],
    [
        -1,  # Giảm
        1   # Tăng
    ],
    default=0  # Ổn định
)

print(data_smooth['label'].value_counts(normalize=True))
print(f"Train size: {data_smooth.shape}")


# data_smooth.reset_index(inplace=True)
# data_smooth = data_smooth.drop(['pct_change', 'pct_change_ma'], axis=1)
data_smooth = data_smooth.dropna()
scaler = StandardScaler()

# Tách dữ liệu thành tập train-validation và tập test
# Giả sử 'Date' là cột chứa ngày tháng và định dạng YYYY-MM-DD
time_eval = 2

eval_months = data_smooth['DATE'].max() - pd.DateOffset(months=time_eval)
#  xác định thời điểm chia eval và test

eval_data = data_smooth[data_smooth['DATE'] >= eval_months].copy()

train_data = data_smooth[data_smooth['DATE'] < eval_months].copy()
#  dữ liệu eval là 1 tháng hiện tại, dùng để xác định xem parameters đã okie chưa


# y_train_lstm = train_data['pct_change'].copy()
# y_val_lstm = eval_data['pct_change'].copy()
train_data = train_data.drop(['pct_change', 'pct_change_ma'], axis=1)

eval_data = eval_data.drop(['pct_change', 'pct_change_ma'], axis=1)

train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data.drop(['DATE','label'],axis=1)), columns=train_data.drop(['DATE','label'],axis=1).columns)
# dùng train để fit_transform và sau đó thì transform eval và test chứ ko dùng fit_transform
#  điều này đảm tính phân phối dữ liệu và cũng không làm leak data future
eval_data_scaled = pd.DataFrame(scaler.transform(eval_data.drop(['DATE','label'],axis=1)), columns=eval_data.drop(['DATE','label'],axis=1).columns)


#  gắn DATE và label vào lại data đã scaled để dễ triển khai
train_data_scaled['DATE'] = train_data['DATE'].to_numpy()
train_data_scaled['label'] = train_data['label'].to_numpy()

eval_data_scaled['DATE'] = eval_data['DATE'].to_numpy()
eval_data_scaled['label'] = eval_data['label'].to_numpy()


#  bỏ DATE và label để hình thành features, lấy label làm test
if 'DATE' in train_data.columns:
    train_data_features = train_data_scaled.drop(['DATE', 'label'], axis=1)
    val_data_features = eval_data_scaled.drop(['DATE', 'label'], axis=1)

else:
    train_data_features = train_data_scaled.drop(['label'], axis=1)
    val_data_features = eval_data_scaled.drop(['label'], axis=1)
    # test_data_features = test_data_scaled.drop(['label'], axis=1)

# Tách nhãn (y)
train_labels = train_data_scaled['label']
val_labels = eval_data_scaled['label']


# Xác định X và y cho tập train, validation và test
X_train = train_data_features
y_train = train_labels

X_val = val_data_features
y_val = val_labels



# Hiển thị kích thước của các tập để kiểm tra
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")



# Hiển thị kích thước của các tập dữ liệu
print(f"Train size: {train_data.shape}")
print(f"Validation size: {eval_data.shape}")

In [ ]:
def calculate_total_profit(y_pred, prices):
    """
    Tính tổng lợi nhuận dựa trên dự đoán của meta-model.

    :param y_true: Nhãn thực tế (không nhất thiết sử dụng trong hàm này nhưng để cho mục đích so sánh).
    :param y_pred: Dự đoán từ meta-model.
    :param prices: Mảng giá tại mỗi điểm thời gian (ví dụ, giá đóng cửa của cổ phiếu).
    :return: Tổng lợi nhuận từ chiến lược.
    """
    profit = 0
    for i in range(1, len(prices)):
        if y_pred[i-1] == 1:  # Dự đoán là tăng
            profit += prices[i] - prices[i-1]  # Lợi nhuận từ việc mua và sau đó bán
        elif y_pred[i-1] == -1:  # Dự đoán là giảm
            profit += prices[i-1] - prices[i]  # Lợi nhuận từ việc bán và sau đó mua lại
    return profit

def caculate_profit(sym, total_profit):
  # Khởi tạo các biến để tính tiền cho forex
  usd_rate = {"EURCHF":1.11, "EURGBP":1.22, "GBPAUD": 0.64, "EURJPY": 0.0066, \
                "USDCHF":1.11, "EURNZD":0.59,  'AUDJPY':0.0066, 'EURCAD':0.72, 'NZDUSD':1,\
                 'AUDCAD':0.72, 'NZDJPY': 0.0066, 'GBPJPY':0.0066, 'USDJPY':0.0066, 'AUDNZD':0.59\
                 , 'USDCAD':0.72 }
  volume = 0.01
  contract_size = 100000
# Sau đó tính tiền
  profit = total_profit*volume*contract_size*usd_rate[sym]
  return profit


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.metrics import accuracy_score
import numpy as np
np.int = int


# LGBM

In [ ]:
import numpy

from lightgbm import LGBMClassifier
split_point_CV = int(len(X_train)*0.90)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train[split_point_CV:]

# Mô hình LightGBM
lgbm_model = LGBMClassifier()
# Thay parameter vào model
para_lgbm =  parameter["LGBM"]
para_lgbm_dict = dict(para_lgbm)
LGBMbase = LGBMClassifier(**para_lgbm_dict)
LGBMbase.fit(X_train,y_train)

y_pred = LGBMbase.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


data_make_test = data[data['DATE'] > time_split].copy()
test_months_ql = data_make_test['DATE'].min() + pd.DateOffset(months=0)
end_months = data_make_test['DATE'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['DATE'] > test_months_ql) & (data_make_test['DATE']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['DATE'],axis=1)), columns=test_data.drop(['DATE'],axis=1).columns)

X_test['DATE'] = test_data['DATE'].to_numpy()

X_test.set_index('DATE', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


# LGBM2

In [ ]:
import numpy
from lightgbm import LGBMClassifier
split_point_CV = int(len(X_train)*0.80)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train[split_point_CV:]

# Mô hình LightGBM
lgbm_model = LGBMClassifier()
# đưa para vào model LGBM2
para_lgbm2 = parameter["LGBM2"]
para_lgbm2_dict = dict(para_lgbm2)
LGBMbase2 = LGBMClassifier(**para_lgbm2_dict)
LGBMbase2.fit(X_train,y_train)

y_pred = LGBMbase2.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


data_make_test = data[data['DATE'] > time_split].copy()
test_months_ql = data_make_test['DATE'].min() + pd.DateOffset(months=0)
end_months = data_make_test['DATE'].min() + pd.DateOffset(months=12)
test_data = data_make_test[(data_make_test['DATE'] > test_months_ql) & (data_make_test['DATE']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['DATE'],axis=1)), columns=test_data.drop(['DATE'],axis=1).columns)

X_test['DATE'] = test_data['DATE'].to_numpy()

X_test.set_index('DATE', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = LGBMbase2.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)


# XGB Model

In [ ]:

y_train_012 = le.fit_transform(y_train)

split_point_CV = int(len(X_train)*0.90)
X_train_BY = X_train[split_point_CV:]
y_train_BY = y_train_012[split_point_CV:]

np.int = int


# tìm được paras tốt nhất thì huấn luyện với đầy đủ data
import tensorflow as tf

para_xgb_dict = dict(parameter["XGB"])
xgbbase = XGBClassifier(**para_xgb_dict)

xgbbase.fit(X_train,y_train_012, verbose=True)


# kiếm tra model đã huấn luyện full data có lời trên tập eval không
y_pred = xgbbase.predict(X_val)
y_pred = le.inverse_transform(y_pred)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)
data_make_test = data[data['DATE'] > time_split].copy()

test_months_ql = data_make_test['DATE'].min() + pd.DateOffset(months=0)
end_months = data_make_test['DATE'].min() + pd.DateOffset(months=4)
test_data = data_make_test[(data_make_test['DATE'] > test_months_ql) & (data_make_test['DATE']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['DATE'],axis=1)), columns=test_data.drop(['DATE'],axis=1).columns)

X_test['DATE'] = test_data['DATE'].to_numpy()

X_test.set_index('DATE', inplace=True)


y_pred = xgbbase.predict(X_test)
y_pred = le.inverse_transform(y_pred)

prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)

# RF model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_xgb_dict = dict(parameter["RF"])
RFbase = RandomForestClassifier(**rf_xgb_dict)
# Thêm các para vào model
RFbase.fit(X_train,y_train)
y_pred = RFbase.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
prices = X_val['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)
data_make_test = data[data['DATE'] > time_split].copy()

test_months_ql = data_make_test['DATE'].min() + pd.DateOffset(months=0)
end_months = data_make_test['DATE'].min() + pd.DateOffset(months=4)
test_data = data_make_test[(data_make_test['DATE'] > test_months_ql) & (data_make_test['DATE']<= end_months)].copy()
X_test = pd.DataFrame(scaler.transform(test_data.drop(['DATE'],axis=1)), columns=test_data.drop(['DATE'],axis=1).columns)

X_test['DATE'] = test_data['DATE'].to_numpy()

X_test.set_index('DATE', inplace=True)
print(f"X_test shape: {X_test.shape}")
print(f"Test size: {test_data.shape}")
y_pred = RFbase.predict(X_test)
prices = X_test['CLOSE'].copy().to_numpy()
total_profit = calculate_total_profit(y_pred,prices)
print(total_profit)

# Các base model vào dùng để stacking model

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import clone
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

base_models = [
    xgbbase,
    RFbase,
    LGBMbase,
    LGBMbase2,
]


meta_features = np.zeros((X_train.shape[0], len(base_models)))
meta_target = np.zeros(X_train.shape[0])


for i, model in enumerate(base_models):
    for train_index, holdout_index in kfold.split(X_train, y_train):
        # Clone model để không làm thay đổi model gốc
        instance = clone(model)
        if isinstance(instance, XGBClassifier):
            y_train_index = le.fit_transform(y_train.copy().iloc[train_index])
        else:
            y_train_index = y_train.copy().iloc[train_index]
        # Huấn luyện mỗi base model trên phần train của fold
        instance.fit(X_train.iloc[train_index], y_train_index)

        # Dự đoán trên phần hold-out và lưu vào meta_features

        y_pred = instance.predict(X_train.iloc[holdout_index])
        if isinstance(instance, XGBClassifier):
            y_pred = le.inverse_transform(y_pred)
        meta_features[holdout_index, i] = y_pred

meta_target = y_train


# Meta model được train bằng CAT, random forest, MLP classifiers

In [ ]:
from catboost import CatBoostClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

catboost_meta_model = CatBoostClassifier(iterations=100, learning_rate=0.01, depth=10, verbose=0)
catboost_meta_model.fit(meta_features, meta_target)


random_forest_meta_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
random_forest_meta_model.fit(meta_features, meta_target)


nn_meta_model = MLPClassifier(hidden_layer_sizes=(200,100,100,50,), activation='relu', solver='adam', max_iter=200, random_state=42)
nn_meta_model.fit(meta_features, meta_target)



# Tình profit của các tháng gần đây nhất

In [ ]:
data_make_test = data[data['DATE'] > time_split].copy()
profit_all = {}
predict_all = {}
for i in range(time_test):
    profit_model =  {}
    predict_model = {}

    time_skips = i
    print('-----------Test tháng thứ ' +str(i+1))
    test_months_ql = data_make_test['DATE'].min() + pd.DateOffset(months=0)
    end_months = data_make_test['DATE'].min() + pd.DateOffset(months=time_skips+1)
    test_data = data_make_test[(data_make_test['DATE'] > test_months_ql) & (data_make_test['DATE']<= end_months)].copy()
    X_test = pd.DataFrame(scaler.transform(test_data.drop(['DATE'],axis=1)), columns=test_data.drop(['DATE'],axis=1).columns)

    X_test['DATE'] = test_data['DATE'].to_numpy()

    X_test.set_index('DATE', inplace=True)

    prices = X_test['CLOSE'].copy().to_numpy()

    meta_features_test = np.zeros((X_test.shape[0], len(base_models)))

    for i, model in enumerate(base_models):
        y_pred_test = model.predict(X_test)
        if isinstance(model, XGBClassifier):
            y_pred_test = le.inverse_transform(y_pred_test)
        meta_features_test[:, i] = y_pred_test



    y_pred = LGBMbase.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--LGB ' + str(total_profit))
    profit_model["LGB"]  = total_profit
    predict_model["LGB"] = y_pred


    y_pred = LGBMbase2.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--LGB2 '+ str(total_profit))
    profit_model["LGB2"]  = total_profit
    predict_model["LGB2"] = y_pred

    y_pred = RFbase.predict(X_test)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--RF '+ str(total_profit))
    profit_model["RF"]  = total_profit
    predict_model["RF"] = y_pred

    y_pred = xgbbase.predict(X_test)
    y_pred = le.inverse_transform(y_pred)
    total_profit = calculate_total_profit(y_pred,prices)
    print(y_pred)
    print('--XGB '+ str(total_profit))
    profit_model["XGB"]  = total_profit
    predict_model["XGB"] = y_pred

    y_pred_finalcat = catboost_meta_model.predict(meta_features_test)
    y_pred_finalcat  =  y_pred_finalcat.flatten()
    total_profit = calculate_total_profit(y_pred_finalcat,prices)
    print(y_pred_finalcat)
    print('--meta cat '+ str(total_profit))
    profit_model["meta cat"]  = total_profit
    predict_model["meta cat"] = y_pred_finalcat

    y_pred_finalrf = random_forest_meta_model.predict(meta_features_test)
    total_profit = calculate_total_profit(y_pred_finalrf,prices)
    print(y_pred_finalrf)
    print('--meta rf '+ str(total_profit))
    profit_model["meta rf"]  = total_profit
    predict_model["meta rf"] = y_pred_finalrf

    y_pred_finalMLP = nn_meta_model.predict(meta_features_test)
    total_profit = calculate_total_profit(y_pred_finalMLP,prices)
    print(y_pred_finalMLP)
    print('--meta MLP '+ str(total_profit))
    profit_model["meta mlp"]  = total_profit
    predict_model["meta mlp"] = y_pred_finalMLP



    print("---------------------------------------------------------------------------")
    print("\n")
    month_target = "Tháng "+ str(time_skips+ 1)
    profit_all[month_target] = profit_model
    predict_all[month_target] = predict_model


In [ ]:
profit_cal = X_test.copy()
profit_cal['PREDICTION'] = y_pred  # Add predictions to DataFrame
profit_cal['PROFIT'] = np.nan  # Initialize profit column
profit_cal['DATE'] = profit_cal.index  # Assign index (date) to the 'DATE' column

for i in range(1, len(profit_cal)):
    if profit_cal['PREDICTION'].iloc[i-1] == 1:  # BUY signal
        print(f'BUY signal on {profit_cal["DATE"].iloc[i]}')  # Display the date of prediction
        profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i] - profit_cal['CLOSE'].iloc[i-1]
    elif profit_cal['PREDICTION'].iloc[i-1] == -1:  # SELL signal
        print(f'SELL signal on {profit_cal["DATE"].iloc[i]}')  # Display the date of prediction
        profit_cal['PROFIT'].iloc[i] = profit_cal['CLOSE'].iloc[i-1] - profit_cal['CLOSE'].iloc[i]
    else:
        print('Hold')

# Calculate total profit
total_profit = profit_cal['PROFIT'].sum()


In [ ]:
print("Lợi nhuận từng ngày:")
print(profit_cal[['DATE', 'PROFIT']])

print("Lợi nhuận tổng cộng:", total_profit)